___

Takes a list of site_codes and returns Daily Values og Gauge Height and Stream Flow from December 1 2015 to December 1 2020. The API url was formatted using the USGS API FWIS Formatting tool. https://waterservices.usgs.gov/rest/IV-Test-Tool.html.

Individual API requests are made for each site in this version of the request. This requires us to do some indexing , and merging, in order to construct a datframe from our JSON formatted response.

In [1]:
import pandas as pd
import requests

url = ["https://waterservices.usgs.gov/nwis/iv/?format=json&sites=","&startDT=2015-12-01T00:00-0600&endDT=2020-12-01T00:00-0600&parameterCd=00060,00065&siteStatus=all"]
site_codes = ['07079300','07081200','07083710','07083200','07083000']

In [2]:
def flow_height_data_pull(site_codes, file_path):

    '''
    file_path: string, target export directory. ex.) "directory/" 
    site_codes: list of strings

    '''
    url = ["https://waterservices.usgs.gov/nwis/iv/?format=json&sites=","&startDT=2015-12-01T00:00-0600&endDT=2020-12-01T00:00-0600&parameterCd=00060,00065&siteStatus=all"]
    df = pd.DataFrame()
    for site in site_codes:

        # API request
        new_url = url[0]+site+url[1]
        res = requests.get(new_url).json()

        # indexing values & make df
        data = res['value']['timeSeries']

        StreamFlowValues = pd.DataFrame(data[0]['values'][0]['value']) 
        GaugeHeightValues = pd.DataFrame(data[1]['values'][0]['value']) 

        # drop extra and rename columns
        try
            StreamFlowValues.drop(columns = ['qualifiers'], inplace = True)
        except: 
            print(site, ' No column qualifiers found, check for missing data.')
            pass


        try:
            GaugeHeightValues.drop(columns = ['qualifiers'], inplace = True)
        except: 
            print(site, ' No column qualifiers found, check for missing data.')
            pass


        try:
            StreamFlowValues.columns =  ['flow','date']
        except: 
            print(site, ' Could not rename column, check for missing data.')
            pass


        try:
            GaugeHeightValues.columns = ['height','date']
        except: 
            print(site, ' Could not rename column, check for missing data.')
            pass


        try:
            temp_df = StreamFlowValues.merge(GaugeHeightValues, left_on='date',right_on='date')
        except:
            print(site, ' MISSING DATA ' ) 
            temp_df = StreamFlowValues

        # create column for site_code 
        temp_df['sitecode'] = site

        # output progress updates
        print(f'Exporting {len(temp_df)} observations from site {site}')    
        temp_df.to_csv(f'{file_path}{site}_height_flow_2015-20.csv')
        df = pd.concat([df, temp_df])

    print(f'\nExporting {len(df)} total observations')    
    df.to_csv(f'{file_path}arkansas_headwaters_height_flow_2015-20.csv')

In [3]:
flow_height_data_pull(site_codes, 'datasets/')

Exporting 5071 observations from site 07079300
Exporting 543 observations from site 07081200
Exporting 3 observations from site 07083710
07083200  No column qualifiers found, check for missing data.
07083200  Could not rename column, check for missing data.
07083200  MISSING DATA 
Exporting 71994 observations from site 07083200
Exporting 911 observations from site 07083000

Exporting 78522 total observations


___

Next time we can :
  1. call all of our sites in one API request at the same time.
  2. wrap functions
      * indexing 
      * cleaning
      * merging
      * naming
  3. Create inputs with defauts to expand the API req:
      * date ranges
      * parameter codes
      * response format 
      * site status
      